# Renew a Slice Reservation


## Setup the Experiment

#### Import FABRIC API

In [5]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Artifact Manager,artifacts.fabric-testbed.net
Token File,/home/alphani/.fabric_token.json
Project ID,220c2d8a-ae05-465a-a98b-8178415e1b30
Bastion Host,bastion.fabric-testbed.net
Bastion Username,sa2994_0000240697
Bastion Private Key File,/home/alphani/work/fabric_config/fabric_bastion_key
Slice Public Key File,/home/alphani/work/fabric_config/slice_key.pub
Slice Private Key File,/home/alphani/work/fabric_config/slice_key


## List project all slices

In [ ]:
import json
from datetime import datetime, timezone, timedelta

# Set end host to now plus 1 day
end_date = (datetime.now(timezone.utc) + timedelta(days=6)).strftime("%Y-%m-%d %H:%M:%S %z")

slices = json.loads(fablib.list_slices(user_only=True, output='json'))

renewed_slices = []
non_renewed_slices = []

for slice_info in slices:
    slice = fablib.get_slice(name=slice_info['name'])
    try:
        old_time = slice.get_lease_end()

        # Convert old_time to datetime if it's a string
        if isinstance(old_time, str):
            old_time = datetime.strptime(old_time, "%Y-%m-%d %H:%M:%S %z")

        # Check if the lease expires in 1 day or less
        if old_time is not None:
            time_remaining = old_time - datetime.now(timezone.utc)
            if time_remaining <= timedelta(days=1):
                slice.renew(end_date)
                new_time = slice.get_lease_end()

                # Convert new_time to datetime if it's a string
                if isinstance(new_time, str):
                    new_time = datetime.strptime(new_time, "%Y-%m-%d %H:%M:%S %z")

                renewed_slices.append((slice.get_name(), slice.get_lease_start(), new_time))
            else:
                non_renewed_slices.append((slice.get_name(), slice.get_lease_start(), old_time))
        else:
            print(f"Slice {slice.get_name()} has no lease end time.")
    except Exception as e:
        print(f"Exception in slice {slice.get_name()}: {e}")

# Print the renewed slices
print("\n=== Renewed Slices ===")
for name, start, new_time in renewed_slices:
    print(f"Slice: {name}, Lease Start: {start}, New Lease End: {new_time}")

# Print the non-renewed slices
print("\n=== Non-Renewed Slices ===")
for name, start, end_time in non_renewed_slices:
    print(f"Slice: {name}, Lease Start: {start}, Lease End: {end_time}")



Retry: 0, Time: 47 sec


ID,823bd6f9-2767-47ec-8363-05986b7b8dc5
Name,Testbed-openvswitch-3
Lease Expiration (UTC),2025-01-13 06:39:13 +0000
Lease Start (UTC),2025-01-11 06:39:13 +0000
Project ID,220c2d8a-ae05-465a-a98b-8178415e1b30
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
5e032499-58e8-4b51-9055-2f606a8a0609,bridge1,2,8,10,default_ubuntu_22,qcow2,gpn-w1.fabric-testbed.net,GPN,ubuntu,2610:e0:a04c:fab2:f816:3eff:fe1a:2767,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@2610:e0:a04c:fab2:f816:3eff:fe1a:2767,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key
cc7ff57c-45ab-48c7-8077-fd9a4ff7096c,bridge2,2,8,10,default_ubuntu_22,qcow2,newy-w1.fabric-testbed.net,NEWY,ubuntu,2001:400:a100:3040:f816:3eff:fe76:b671,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@2001:400:a100:3040:f816:3eff:fe76:b671,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key
110f88ed-60b2-4cad-9018-e177f52dca4f,prin-bridge1-1,2,8,10,default_ubuntu_22,qcow2,prin-w2.fabric-testbed.net,PRIN,ubuntu,2620:c4:0:180:f816:3eff:fe73:4106,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@2620:c4:0:180:f816:3eff:fe73:4106,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key
919ce062-c4a7-4dad-b679-900f01c67ec3,prin-bridge1-2,2,8,10,default_ubuntu_22,qcow2,prin-w2.fabric-testbed.net,PRIN,ubuntu,2620:c4:0:180:f816:3eff:feb4:c4e,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@2620:c4:0:180:f816:3eff:feb4:c4e,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key
59a847ac-b0bd-4a1d-a79a-a6859bf1cf33,sri-bridge2-1,2,8,10,default_ubuntu_22,qcow2,sri-w2.fabric-testbed.net,SRI,ubuntu,192.5.67.91,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@192.5.67.91,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key
7a0d1bb8-a7ab-4721-a420-7144348f55c2,sri-bridge2-2,2,8,10,default_ubuntu_22,qcow2,sri-w2.fabric-testbed.net,SRI,ubuntu,192.5.67.189,Active,,ssh -i /home/alphani/work/fabric_config/slice_key -F /home/alphani/work/fabric_config/ssh_config ubuntu@192.5.67.189,/home/alphani/work/fabric_config/slice_key.pub,/home/alphani/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
60f24181-7262-496b-8f12-14044e6f6cb1,net-br1-1,L2,L2STS,None,None,None,Active,
3aa47158-c4cb-42ca-956e-5c4c3b930906,net-br1-2,L2,L2STS,None,None,None,Active,
ddcf0bf7-f2f1-487e-89b6-5a2bd34e6818,net-br1-br2,L2,L2STS,None,None,None,Active,
69a2d500-6d55-4d20-83d1-af11404d697b,net-br2-1,L2,L2STS,None,None,None,Active,
7e73dd60-edd7-432e-bbdd-cc95b830de60,net-br2-2,L2,L2STS,None,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
bridge1-nic_local_1-p1,p1,bridge1,net-br1-1,100,config,,32:A2:17:D7:38:7B,enp9s0,enp9s0,fe80::30a2:17ff:fed7:387b,6,HundredGigE0/0/0/5
bridge1-nic_local_2-p1,p1,bridge1,net-br1-2,100,config,,2E:5E:F3:E9:1B:AE,enp8s0,enp8s0,fe80::2c5e:f3ff:fee9:1bae,6,HundredGigE0/0/0/5
bridge1-nic_local_3-p1,p1,bridge1,net-br1-br2,100,config,,2A:53:67:68:DA:B1,enp7s0,enp7s0,fe80::2853:67ff:fe68:dab1,6,HundredGigE0/0/0/5
bridge2-nic_local_3-p1,p1,bridge2,net-br2-1,100,config,,06:21:1C:D3:8B:F9,enp9s0,enp9s0,fe80::421:1cff:fed3:8bf9,4,HundredGigE0/0/0/5
bridge2-nic_local_1-p1,p1,bridge2,net-br2-2,100,config,,02:9C:D1:81:80:D5,enp7s0,enp7s0,fe80::9c:d1ff:fe81:80d5,4,HundredGigE0/0/0/5
bridge2-nic_local_2-p1,p1,bridge2,net-br1-br2,100,config,,06:1E:52:DC:0C:DB,enp8s0,enp8s0,fe80::41e:52ff:fedc:cdb,4,HundredGigE0/0/0/5
prin-bridge1-1-nic_local-p1,p1,prin-bridge1-1,net-br1-1,100,config,,0A:67:13:80:B7:6D,enp7s0,enp7s0,10.10.10.1,4,HundredGigE0/0/0/9
prin-bridge1-2-nic_local-p1,p1,prin-bridge1-2,net-br1-2,100,config,,0A:CE:97:3C:EC:AF,enp7s0,enp7s0,10.10.10.2,4,HundredGigE0/0/0/9
sri-bridge2-1-nic_local-p1,p1,sri-bridge2-1,net-br2-1,100,config,,02:87:0F:34:90:FD,enp7s0,enp7s0,10.10.10.3,4,HundredGigE0/0/0/9
sri-bridge2-2-nic_local-p1,p1,sri-bridge2-2,net-br2-2,100,config,,06:5C:64:37:8B:68,enp7s0,enp7s0,10.10.10.4,4,HundredGigE0/0/0/9



Time to print interfaces 72 seconds

=== Renewed Slices ===
Slice: Testbed-openvswitch-3, Lease Start: 2025-01-11 06:39:13 +0000, New Lease End: 2025-01-13 06:39:13+00:00

=== Non-Renewed Slices ===
Slice: Testbed-openvswitch-4, Lease Start: 2025-01-11 14:21:56 +0000, Lease End: 2025-01-17 02:25:23+00:00
